Given two words (beginWord and endWord), and a dictionary's word list, find the length of shortest transformation sequence from beginWord to endWord, such that:

Only one letter can be changed at a time.
Each transformed word must exist in the word list. Note that beginWord is not a transformed word.
For example,

Given:

beginWord = "hit"

endWord = "cog"

wordList = ["hot","dot","dog","lot","log","cog"]

As one shortest transformation is "hit" -> "hot" -> "dot" -> "dog" -> "cog",

return its length 5.

Note:

Return 0 if there is no such transformation sequence.

All words have the same length.

All words contain only lowercase alphabetic characters.

You may assume no duplicates in the word list.

You may assume beginWord and endWord are non-empty and are not the same.


## Solution

The steps from word to word can be modeled as a graph.  So, the first step is to build a graph from our word list.  We then need to do a search through the graph to find paths.

We have the option of searching BF or DF.  Both should find us a solition, eventually.  However, two-ended BFS will solve it fastest.

In [5]:
from copy import deepcopy

def word_distance(a, b):
    dist = 0
    for aa, bb in zip(a,b):
        if not aa == bb:
            dist += 1
    return dist


def build_graph(word_list):
    one_step_graph = {}
    for i, word1 in enumerate(word_list[:-1]):
        for word2 in word_list[i+1:]:
            if not word1 in one_step_graph:
                one_step_graph[word1] = set()
            if not word2 in one_step_graph:
                one_step_graph[word2] = set()
            if word_distance(word1, word2) == 1:
                one_step_graph[word1].add(word2)
                one_step_graph[word2].add(word1)
    return one_step_graph

In [36]:
def bfs(left_paths, explored_left, end_word, gr):
    next_paths = []
    
    if len(left_paths) == 0:
        return None
    
    for curr_path in left_paths:
        for n_word in gr[curr_path[-1]]:
            next_path = deepcopy(curr_path) + [n_word]
            if n_word == end_word:
                return next_path
            elif n_word in explored_left:
                continue
            else:
                explored_left.add(n_word)
                next_paths.append(next_path)
    return bfs(next_paths, explored_left, end_word, gr)


def word_ladder(begin_word, end_word, word_list):
    if begin_word == end_word:
        return [begin_word]
    else:
        gr = build_graph(word_list + [begin_word])
        res = bfs([[begin_word]], set(), end_word, gr)
        
    if not res:
        print("No solution")
    else:
        return res
    
    
l = ["hot","dot","dog","lot","log","cog"]
begin_word = "lat"
end_word = "cog"
sol = word_ladder(begin_word, end_word, l)
print(sol)

l = ['a', 'b', 'c']
begin_word = "a"
end_word = "c"
sol = word_ladder(begin_word, end_word, l)
print(sol)

['lat', 'lot', 'log', 'cog']
['a', 'c']


## Two ended BFS

In [37]:
def two_sided_bfs(left_paths, explored_left, right_paths, explored_right, gr):
    next_left_paths = []
    next_right_paths = []
    
    if len(left_paths) == 0 and len(right_paths) == 0:
        return None
    
    #advance left side
    for curr_path in left_paths:
        for n_word in gr[curr_path[-1]]:
            next_left_path = deepcopy(curr_path) + [n_word]
            
            if n_word in explored_right.keys():
                r_path, idx = explored_right[n_word]
                r_path = r_path[:idx]
                r_path.reverse()
                return next_left_path + r_path
            elif n_word in explored_left:
                # drop path
                continue
            else:
                explored_left[n_word] = (next_left_path, len(next_left_path) - 1)
                next_left_paths.append(next_left_path)
                
    # advance right side
    for curr_path in right_paths:
        for n_word in gr[curr_path[-1]]:
            next_right_path = deepcopy(curr_path) + [n_word]
            
            if n_word in explored_left.keys():
                l_path, idx = explored_left[n_word]
                next_right_path.reverse()
                return l_path[:idx] + next_right_path
            elif n_word in explored_right:
                # drop path
                continue
            else:
                explored_right[n_word] = (next_right_path, len(next_right_path) - 1)
                next_right_paths.append(next_right_path)
    
    
    return two_sided_bfs(next_left_paths, explored_left, next_right_paths, explored_right, gr)


def word_ladder_2(begin_word, end_word, word_list):
    if begin_word == end_word:
        return [begin_word]
    else:
        gr = build_graph(word_list + [begin_word])
        start_left_path = [begin_word]
        explored_left = {begin_word : (start_left_path, 0)}
        start_right_path = [end_word]
        explored_right = {end_word : (start_right_path, 0)}
        res = two_sided_bfs(
            [start_left_path],
            explored_left,
            [start_right_path],
            explored_right,
            gr)
        
    if not res:
        print("No solution")
    else:
        return res

In [38]:
l = ["hot","dot","dog","lot","log","cog"]
begin_word = "lat"
end_word = "cog"
sol = word_ladder_2(begin_word, end_word, l)
print(sol)

['lat', 'lot', 'log', 'cog']
